# Анализ резултатов работы прототипа OCR. Детекция OpenVINO. Распознавание PaddleOCR 
- Автор: Кирилл Киселев
- Дата начала: 28.04.2022
- Описание исследования: после работы настройки препроцессинга на прототипе и замены модели детекции на модель из фреймфорка OpenVIN проведено тестирование на 203 сканах товарных накладных. 10 сканов были отсеяны на стадии проверки блюра и ортогонального положения. Остальные 193 скана пропустим через анализ полученных результатов.

## Загрузка библиотек

In [1]:
import os
import pandas as pd
import cv2

## Глобальные переменные

In [2]:
!pwd

/home/kirillk/PycharmProjects/useful_notebooks/ocr


In [44]:
# CSV_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results.csv'
# CSV_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_1.csv'
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_3.txt'
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_4.txt'
TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_5.txt'
SAVE_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results'
DATA_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/test_cases/'

## Загрузка данных и подготовка датасета

In [45]:
filename_list = []
results_list = []

with open(TXT_PATH, "r") as file:
    for num, line in enumerate(file):
        if 'jpg' in line:
            line = line.split(' ')[-1].rstrip()
            filename_list.append(line)
        else:
            line = line.split(' ', maxsplit=3)[-1].rstrip()
            results_list.append(line)
            
results_list = results_list[0:-1]

In [46]:
filename_list

['108.jpg',
 '134.jpg',
 '145.jpg',
 '020.jpg',
 '172.jpg',
 '030.jpg',
 '044.jpg',
 '072.jpg',
 '183.jpg',
 '088.jpg',
 '192.jpg',
 '191.jpg',
 '122.jpg',
 '029.jpg',
 '144.jpg',
 '109.jpg',
 '049.jpg',
 '037.jpg',
 '138.jpg',
 '200.jpg',
 '189.jpg',
 '114.jpg',
 '195.jpg',
 '155.jpg',
 '093.jpg',
 '010.jpg',
 '148.jpg',
 '133.jpg',
 '199.jpg',
 '124.jpg',
 '032.jpg',
 '152.jpg',
 '171.jpg',
 '177.jpg',
 '075.jpg',
 '038.jpg',
 '000.jpg',
 '173.jpg',
 '126.jpg',
 '035.jpg',
 '003.jpg',
 '062.jpg',
 '094.jpg',
 '184.jpg',
 '004.jpg',
 '083.jpg',
 '164.jpg',
 '025.jpg',
 '101.jpg',
 '028.jpg',
 '076.jpg',
 '146.jpg',
 '067.jpg',
 '052.jpg',
 '087.jpg',
 '018.jpg',
 '051.jpg',
 '147.jpg',
 '187.jpg',
 '047.jpg',
 '115.jpg',
 '140.jpg',
 '079.jpg',
 '061.jpg',
 '096.jpg',
 '007.jpg',
 '011.jpg',
 '023.jpg',
 '120.jpg',
 '084.jpg',
 '065.jpg',
 '098.jpg',
 '130.jpg',
 '175.jpg',
 '022.jpg',
 '009.jpg',
 '069.jpg',
 '014.jpg',
 '054.jpg',
 '019.jpg',
 '056.jpg',
 '008.jpg',
 '017.jpg',
 '16

In [47]:
results_list

["{'id': 0, 'barcode': '2860587076540', 'doc_type': 'Товарная накладная', 'doc_number': '6017110800', 'error_list': []}",
 "{'id': 1, 'barcode': '2860592637279', 'doc_type': 'товарная гнакладная', 'doc_number': '6017253079', 'error_list': []}",
 "{'id': 2, 'barcode': '2860601366206', 'doc_type': 'Товарнаяі накладная', 'doc_number': '6047415447', 'error_list': []}",
 "{'id': 3, 'barcode': '2810094798084', 'doc_type': 'Товарная накладная', 'doc_number': '6010358825', 'error_list': []}",
 "{'id': 4, 'barcode': '2860614983032', 'doc_type': 'товарная накладная', 'doc_number': '6017580463', 'error_list': []}",
 "{'id': 5, 'barcode': '2810095110823', 'doc_type': 'Товарная накладная', 'doc_number': '6010496453', 'error_list': []}",
 "{'id': 6, 'barcode': '2860350653671', 'doc_type': 'Товарная накладная', 'doc_number': '6010495761', 'error_list': []}",
 "{'id': 7, 'barcode': '2860356028688', 'doc_type': 'Товарная накладная', 'doc_number': '6010580640', 'error_list': []}",
 "{'id': 8, 'barcode':

In [48]:
df = pd.DataFrame()

for num, _ in enumerate(results_list):
    q = eval(results_list[num].replace('\\', ''))
    if q['error_list'] == []:
        q['error_list'] = 0
    else:
        q['error_list'] = 1
    
    res_df = pd.DataFrame(q, index=[0])
    df = pd.concat([df, res_df])

In [49]:
df = df.reset_index(drop=True)

In [50]:
df['filename'] = filename_list

In [51]:
df = df[['id', 'filename', 'barcode', 'doc_type', 'doc_number', 'error_list']]

In [52]:
df.head()

,id,filename,barcode,doc_type,doc_number,error_list
0,0,108.jpg,2860587076540,Товарная накладная,6017110800,0
1,1,134.jpg,2860592637279,товарная гнакладная,6017253079,0
2,2,145.jpg,2860601366206,Товарнаяі накладная,6047415447,0
3,3,020.jpg,2810094798084,Товарная накладная,6010358825,0
4,4,172.jpg,2860614983032,товарная накладная,6017580463,0


## Визуализация

## Анализ данных

### Необходимые функции

In [53]:
def not_numeric_in_colunm(col):
    not_numeric = {}
    for i, value in enumerate(col):
        if col[i].isnumeric() == False:
            not_numeric[i] = value
    return not_numeric

In [54]:
def len_of_numbers(col):
    length_dict = {}
    for i, value in enumerate(col):
        if value.isnumeric():
            if len(value) not in length_dict.keys():
                length_dict[len(value)] = 1
            else:
                length_dict[len(value)] += 1
    return length_dict

### Error list = 1

Если в столбце **error_list** 0, значит изображение прошло проверку детектором блюра и предварительным распознавателем текста (определение ортогонального положения). 

Если в этом столбце 1, значит имеет место одна или две ошибки. следовательно изображение не будет распознано. Необходимо оценить качество работы детектора блюра и ортогонального оперделителя. 

In [55]:
df['error_list'].value_counts()

0    193
1     10
Name: error_list, dtype: int64

Итого не будет распознано **9** изображений. Оценим их.

In [56]:
error_df = df[df['error_list'] == 1]
error_df

,id,filename,barcode,doc_type,doc_number,error_list
22,22,195.jpg,None,None,None,1
23,23,155.jpg,2860609887666,Товарнам кнакладная,8817053744,1
60,60,115.jpg,2860611107011,None,None,1
83,83,168.jpg,2860611147581,None,0517850744,1
90,90,112.jpg,2860609826856,None,None,1
100,100,159.jpg,2860610411164,None,None,1
123,123,166.jpg,2860611250205,None,0011033072,1
166,166,170.jpg,None,None,None,1
178,178,157.jpg,2860610590685,None,None,1
185,185,174.jpg,None,None,None,1


**Вывод**

Можно сказать, что деткетор блюра позволил отсеять размытые изображения, их загрузка в SAP нежелательная, поскольку большая часть информации в них потеряна.

### Error list = 0

In [57]:
good_df = df[df['error_list'] == 0]
good_df.head(10)

,id,filename,barcode,doc_type,doc_number,error_list
0,0,108.jpg,2860587076540,Товарная накладная,6017110800,0
1,1,134.jpg,2860592637279,товарная гнакладная,6017253079,0
2,2,145.jpg,2860601366206,Товарнаяі накладная,6047415447,0
3,3,020.jpg,2810094798084,Товарная накладная,6010358825,0
4,4,172.jpg,2860614983032,товарная накладная,6017580463,0
5,5,030.jpg,2810095110823,Товарная накладная,6010496453,0
6,6,044.jpg,2860350653671,Товарная накладная,6010495761,0
7,7,072.jpg,2860356028688,Товарная накладная,6010580640,0
8,8,183.jpg,2860618787148,товарная накладная,60417651087,0
9,9,088.jpg,2810078644574,Товарная накладная,6008222134,0


In [58]:
good_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 0 to 202
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          193 non-null    int64 
 1   filename    193 non-null    object
 2   barcode     193 non-null    object
 3   doc_type    191 non-null    object
 4   doc_number  185 non-null    object
 5   error_list  193 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 10.6+ KB


In [59]:
good_df  = good_df.fillna('None')

In [60]:
good_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 0 to 202
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          193 non-null    int64 
 1   filename    193 non-null    object
 2   barcode     193 non-null    object
 3   doc_type    193 non-null    object
 4   doc_number  193 non-null    object
 5   error_list  193 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 10.6+ KB


In [61]:
good_df = good_df.reset_index(drop=True)

**Вывод**

Далее будем работать с этим датасетом. Отфильтрованные изображением считаем истинно-положительными.

### Barcode

In [62]:
bad_barcode_df = good_df[good_df['barcode'] == 'None'].reset_index(drop=True)
bad_barcode_df

,id,filename,barcode,doc_type,doc_number,error_list


In [63]:
barcode_not_numeric = not_numeric_in_colunm(good_df['barcode'])
barcode_not_numeric

{}

In [64]:
length_of_barcode_numbers = len_of_numbers(good_df['barcode'])
length_of_barcode_numbers

{13: 193}

In [65]:
length_of_barcode_numbers[13] / len(good_df)

1.0

In [66]:
bad_barcode_list = list(bad_barcode_df['filename'])
bad_barcode_list

[]

#### Вывод 
В столбце со штрих-кодами 1 пропущенное значение **None**. Остальные значения соответствуют десятизначным штрих-кодам. Итого можно считать распознанными **99,5%** штрих-кодов.

### Doc type

In [67]:
good_df['doc_type'].unique()

array(['Товарная накладная', 'товарная гнакладная', 'Товарнаяі накладная',
       'товарная накладная', 'Товарная гнакладная', 'Товарная некладная',
       'None', 'Товерная наюладная', 'Товарная накледная',
       'Товерная накладная', 'товарнаят накладная', 'Товарная накпадная',
       'Товаркая накладная', 'Товарная макладная', 'говарнаяі накледная',
       'Товарная нактадная', 'Товарная інакладная', 'товарная наюладная',
       'Товарная накладнея', 'товерная накладнал', 'Тонарная накладная',
       'товарная накладнаяі'], dtype=object)

In [68]:
good_df['doc_type'].value_counts()

Товарная накладная     144
товарная накладная      25
Товарная гнакладная      4
None                     2
Товарная макладная       1
Тонарная накладная       1
товерная накладнал       1
Товарная накладнея       1
товарная наюладная       1
Товарная інакладная      1
Товарная нактадная       1
говарнаяі накледная      1
Товарная накпадная       1
Товаркая накладная       1
товарная гнакладная      1
товарнаят накладная      1
Товерная накладная       1
Товарная накледная       1
Товерная наюладная       1
Товарная некладная       1
Товарнаяі накладная      1
товарная накладнаяі      1
Name: doc_type, dtype: int64

In [69]:
1 - len(good_df[good_df['doc_type'] == 'None']) / len(df)

0.9901477832512315

In [70]:
bad_doc_type_df = good_df[good_df['doc_type'] == 'None']
bad_doc_type_df.head()

,id,filename,barcode,doc_type,doc_number,error_list
31,33,177.jpg,2860617614391,None,6017644991,0
49,51,146.jpg,2860601473409,None,8817416140,0


In [71]:
bad_doc_type_list = list(bad_doc_type_df['filename'])
bad_doc_type_list

['177.jpg', '146.jpg']

#### Вывод 
В столбце с типами документов 3 пропущенных значения **None**. Также в этом столбце есть почти распознанные данные, например **Товарная накладнея**, **товарная накладнаяi** и другие. В почти распознанных данных допущено по 1 ошибке, так что, можно считать их проходящими по порогу распознавания. Итого качественно распознано 190 типов документов из 193 примеров, или всего **98,5%**.

### Doc number

In [72]:
bad_doc_number = good_df[good_df['doc_number'] == 'None']
bad_doc_number.head(10)

,id,filename,barcode,doc_type,doc_number,error_list
24,26,148.jpg,2860602715096,Товарная накладная,None,0
43,45,083.jpg,2810096516426,Товарная накладная,None,0
48,50,076.jpg,2810096516365,Товарная накладная,None,0
59,62,079.jpg,2810097409826,Товарная накладная,None,0
67,70,065.jpg,2810097409826,Товарная накладная,None,0
105,111,165.jpg,2860611233796,говарнаяі накледная,None,0
106,112,113.jpg,2860597420210,Товарная гнакладная,None,0
157,164,081.jpg,2810096516365,Товарная накладная,None,0


In [73]:
bad_doc_number_df = good_df[good_df['doc_number'].str.isnumeric() == False]
bad_doc_number_df.tail(10)

,id,filename,barcode,doc_type,doc_number,error_list
43,45,083.jpg,2810096516426,Товарная накладная,None,0
44,46,164.jpg,2860611204697,Товарная накладная,B047539166,0
48,50,076.jpg,2810096516365,Товарная накладная,None,0
59,62,079.jpg,2810097409826,Товарная накладная,None,0
67,70,065.jpg,2810097409826,Товарная накладная,None,0
93,98,125.jpg,2860582098769,Товарная гнакладная,6Q17073063,0
105,111,165.jpg,2860611233796,говарнаяі накледная,None,0
106,112,113.jpg,2860597420210,Товарная гнакладная,None,0
107,113,180.jpg,2860618473492,товарная накладная,9AOkyKGKK2,0
157,164,081.jpg,2810096516365,Товарная накладная,None,0


In [74]:
len(bad_doc_number_df)

11

In [75]:
bad_doc_number_df['doc_number'].value_counts()

None          8
B047539166    1
6Q17073063    1
9AOkyKGKK2    1
Name: doc_number, dtype: int64

In [76]:
doc_number_not_numeric = not_numeric_in_colunm(good_df['doc_number'])
doc_number_not_numeric

{24: 'None',
 43: 'None',
 44: 'B047539166',
 48: 'None',
 59: 'None',
 67: 'None',
 93: '6Q17073063',
 105: 'None',
 106: 'None',
 107: '9AOkyKGKK2',
 157: 'None'}

In [77]:
length_of_doc_numbers = len_of_numbers(good_df['doc_number'])
length_of_doc_numbers

{10: 180, 11: 2}

In [78]:
for i in good_df['doc_number']:
    if len(i) > 10:
        print(i)

60417651087
80175397303


In [79]:
length_of_doc_numbers[10]/len(good_df)

0.9326424870466321

In [80]:
bad_doc_number_list = list(bad_doc_number_df['filename'])
bad_doc_number_list

['148.jpg',
 '083.jpg',
 '164.jpg',
 '076.jpg',
 '079.jpg',
 '065.jpg',
 '125.jpg',
 '165.jpg',
 '113.jpg',
 '180.jpg',
 '081.jpg']

#### Вывод 
В столбце с номерами документов 124 пропущенных значения. В них фигурируют либо **None** либо нечисловые значения, например **Konmueerue**. Качественно распознано 79 номеров документов из 203 примеров, или всего 38,9%.

## Результат исследования

In [81]:
bad_lists_union = sorted(list(set().union(bad_barcode_list, bad_doc_type_list, bad_doc_number_list)))

In [82]:
len(bad_lists_union)

13

In [83]:
len(df) - len(bad_lists_union)

190

In [84]:
1 -len(bad_lists_union) / len(df)

0.9359605911330049

## Общий вывод

Можно считать качественно распознанными 189 документов из 203, что составляет **93%** из тестовых данных.